<a href="https://colab.research.google.com/github/Omegaraspati/omegaraspati/blob/master/balance_dataset_ep200_Adamax.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import vgg19
# from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.optimizers import Adamax
from tensorflow.keras.metrics import categorical_crossentropy

from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model

from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
train_path  = '/content/drive/MyDrive/Dataset/Train'
valid_path  = '/content/drive/MyDrive/Dataset/Valid'
test_path  = '/content/drive/MyDrive/Dataset/Test'

In [ ]:
train_datagen = ImageDataGenerator(
                             rescale=1./255,
                             rotation_range=40,
                             width_shift_range=0.2,
                             height_shift_range=0.2,
                             shear_range=0.2,
                             zoom_range=0.2,
                             horizontal_flip=True,
                             fill_mode = 'nearest')

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory(train_path,
                                              target_size=(224,224),
                                              batch_size=16,
                                              class_mode = 'categorical')

valid_generator = val_datagen.flow_from_directory(valid_path,
                                              target_size=(224,224),
                                              batch_size=16,
                                              class_mode = 'categorical')

test_generator = test_datagen.flow_from_directory(test_path,
                                              target_size=(224,224),
                                              batch_size=1,
                                              class_mode = 'categorical',
                                              shuffle=False)

Found 3517 images belonging to 2 classes.
Found 1005 images belonging to 2 classes.
Found 490 images belonging to 2 classes.


In [ ]:
nama_class = train_generator.class_indices
print(nama_class)

{'Normal': 0, 'Osteoporosis': 1}


In [ ]:
li = list(nama_class.keys())
print(li)

['Normal', 'Osteoporosis']


In [ ]:
jumlah_class = len(nama_class)
print(jumlah_class)

2


In [ ]:
import pandas as pd

IMG_WIDTH=224
IMG_HEIGHT=224
IMG_DIM = (IMG_WIDTH, IMG_HEIGHT, 3)

vgg_model = vgg19.VGG19(weights = "imagenet", include_top=False, input_shape = IMG_DIM)

#inisialisasi layer terakhir pada block5 sbg output
for layer in vgg_model.layers[:-15]:
    layer.trainable = False

x = vgg_model.output
x = GlobalAveragePooling2D()(x)
x = Flatten()(x)
x = Dense(units=4096, activation='relu')(x)
x = Dense(units=4096, activation='relu')(x)
output  = Dense(units=2, activation='softmax')(x)
model = Model(vgg_model.input, output)


model.summary()

layers = [(layer, layer.name, layer.trainable) for layer in vgg_model.layers]
pd.DataFrame(layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable'])

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

,Layer Type,Layer Name,Layer Trainable
0,<keras.engine.input_layer.InputLayer object at...,input_2,False
1,<keras.layers.convolutional.conv2d.Conv2D obje...,block1_conv1,False
2,<keras.layers.convolutional.conv2d.Conv2D obje...,block1_conv2,False
3,<keras.layers.pooling.max_pooling2d.MaxPooling...,block1_pool,False
4,<keras.layers.convolutional.conv2d.Conv2D obje...,block2_conv1,False
5,<keras.layers.convolutional.conv2d.Conv2D obje...,block2_conv2,False
6,<keras.layers.pooling.max_pooling2d.MaxPooling...,block2_pool,False
7,<keras.layers.convolutional.conv2d.Conv2D obje...,block3_conv1,True
8,<keras.layers.convolutional.conv2d.Conv2D obje...,block3_conv2,True
9,<keras.layers.convolutional.conv2d.Conv2D obje...,block3_conv3,True


In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=Adamax(lr=0.0001),
              metrics=['accuracy'])

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adamax.py:95: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adamax, self).__init__(name, **kwargs)


In [ ]:
from tensorflow.keras.callbacks import Callback, ModelCheckpoint, TensorBoard
import time
import datetime

now = datetime.datetime.now
t = now()

NAME = "logs{}".format(int(time.time()))
tensorboard = TensorBoard(log_dir='/content/drive/MyDrive/Hasil Training/{}'.format(NAME))
checkpointer = ModelCheckpoint(filepath='/content/drive/MyDrive/Hasil Training/Adamax200.hdf5', monitor='val_accuracy', save_best_only=True, verbose=1)


step_size_train = train_generator.n//train_generator.batch_size
step_size_valid = valid_generator.n//valid_generator.batch_size

history = model.fit_generator(train_generator, steps_per_epoch=step_size_train,
                   validation_data=valid_generator, validation_steps=step_size_valid,
                   epochs=200,
                   verbose=1,
                   callbacks=[checkpointer, tensorboard])

print('Training time: %s' % (now() - t))

<ipython-input-24-384d4da66c72>:16: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_generator, steps_per_epoch=step_size_train,


Epoch 1/200
219/219 [==============================] - ETA: 0s - loss: 0.6453 - accuracy: 0.6424
Epoch 1: val_accuracy improved from -inf to 0.61996, saving model to /content/drive/MyDrive/Hasil Training/Adamax200.hdf5
219/219 [==============================] - 1629s 7s/step - loss: 0.6453 - accuracy: 0.6424 - val_loss: 0.7278 - val_accuracy: 0.6200
Epoch 2/200
219/219 [==============================] - ETA: 0s - loss: 0.5638 - accuracy: 0.7372
Epoch 2: val_accuracy improved from 0.61996 to 0.65827, saving model to /content/drive/MyDrive/Hasil Training/Adamax200.hdf5
219/219 [==============================] - 80s 364ms/step - loss: 0.5638 - accuracy: 0.7372 - val_loss: 0.6352 - val_accuracy: 0.6583
Epoch 3/200
219/219 [==============================] - ETA: 0s - loss: 0.5332 - accuracy: 0.7498
Epoch 3: val_accuracy improved from 0.65827 to 0.68851, saving model to /content/drive/MyDrive/Hasil Training/Adamax200.hdf5
219/219 [==============================] - 78s 355ms/step - loss: 0.53

In [ ]:
# Retrieve a list of accuracy results on training and test data
# sets for each training epoch
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

# Retrieve a list of list results on training and test data
# sets for each training epoch
loss = history.history['loss']
val_loss = history.history['val_loss']

# Get number of epochs
epochs = range(len(acc))

# Plot training and validation accuracy per epoch
plt.plot(epochs, acc, label = "training")
plt.plot(epochs, val_acc, label = "validation")
plt.legend(loc="upper left")
plt.grid(False)
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and validation accuracy VGG19')

plt.figure()

# Plot training and validation loss per epoch
plt.plot(epochs, loss, label = "training")
plt.plot(epochs, val_loss, label = "validation")
plt.legend(loc="upper right")
plt.grid(False)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and validation loss VGG19')

In [ ]:
def save_model_result(model):
	'''
		-- model : compiled model on the given data.
	'''

	# Extract model result data to separate data vectors
	data_val_acc = list(model.history.history['val_accuracy'])
	data_acc = list(model.history.history['accuracy'])
	data_val_loss = list(model.history.history['val_loss'])
	data_loss = list(model.history.history['loss'])

	# Convert Model result data to dataframe by using dictionary dat structure
	d = {}
	d['val_accuracy'] = data_val_acc
	d['accuracy'] = data_acc
	d['val_loss'] = data_val_loss
	d['loss'] = data_loss

	df = pd.DataFrame(d)
	print(df)

	# Get a current timestamp
	timestamp = str(datetime.datetime.now()).replace(":","-")[:-10].replace(' ', '_')
	filename = '/content/drive/MyDrive/Hasil Training/Adamax200{}.csv'.format(timestamp)
	df.to_csv(filename, encoding='utf-8')

	print('\n\nResult data is saved as file: {}'.format(filename))

	return df

In [ ]:
import datetime
import pandas as pd

model = model
df = save_model_result(model)

In [ ]:
test_dir = '/content/drive/MyDrive/Dataset/Test'

In [ ]:
test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
test_generator = test_datagen.flow_from_directory(directory=test_dir,
                                                  target_size=(224,224),
                                                  batch_size=1,
                                                  class_mode='categorical',
                                                  shuffle=False)

In [ ]:
nama_class = test_generator.class_indices
print(nama_class)

In [ ]:
li = list(nama_class.keys())
print(li)

In [ ]:
jum_classes = test_generator.num_classes
jum_classes

In [ ]:
steps_test = test_generator.n / test_generator.batch_size
steps_test

In [ ]:
from sklearn.datasets import load_files
from keras.utils import np_utils
import numpy as np

def load_dataset(path):
    data = load_files(path)
    paths = np.array(data['filenames'])
    targets = np_utils.to_categorical(np.array(data['target']))
    return paths, targets

test_files, test_targets = load_dataset('/content/drive/MyDrive/Dataset/Test')

In [ ]:
from tensorflow.keras.preprocessing import image
# from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img

from tqdm import tqdm

def path_to_tensor(img_path):
    #load image, resize ke ukuran 224,224
    img = image.load_img(img_path, target_size=(224, 224))
    #convert gambar jadi 3D tensor (224,224,3)
    x = image.img_to_array(img).astype(np.float32)/255
    # x = image.img_to_array(img)
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

test_tensors = (paths_to_tensor(test_files))

In [ ]:
from tensorflow.keras.models import load_model
model = load_model('/content/drive/MyDrive/Hasil Training/Adamax200.hdf5')
result = print('\nTesting loss: {:.4f}\nTesting accuracy: {:.4f}'.format(*model.evaluate(test_tensors, test_targets, batch_size=test_generator.batch_size)))

from sklearn.metrics import confusion_matrix
import numpy as np

cm_labels = ['Normal','Osteoporosis']

cm = confusion_matrix(np.argmax(test_targets, axis=1),
                      np.argmax(model.predict(test_tensors), axis=1))
plt.imshow(cm, cmap=plt.cm.Blues)
plt.colorbar()
indexes = np.arange(len(cm_labels))
for i in indexes:
    for j in indexes:
        plt.text(j, i, cm[i, j])
plt.xticks(indexes, cm_labels, rotation=90)
plt.xlabel('Predicted label')
plt.yticks(indexes, cm_labels)
plt.ylabel('True label')
plt.title('Confusion matrix')
plt.grid(False)
plt.show()

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from tensorflow.keras.models import load_model

from tensorflow.keras import backend as K

test_dir = '/content/drive/MyDrive/Dataset/Test'

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(224, 224),
        batch_size= 1,
        class_mode='categorical',
        shuffle=False)

model_vgg19 = load_model('/content/drive/MyDrive/Hasil Training/Adamax200.hdf5')

#Confusion Matrix and Classification Report
Y_pred_vgg19 = model_vgg19.predict_generator(test_generator, test_generator.samples // test_generator.batch_size)
y_pred_vgg19 = np.argmax(Y_pred_vgg19, axis=1)


print('Confusion Matrix VGG19')
print(confusion_matrix(test_generator.classes, y_pred_vgg19))
print('Classification Report')
target_names = ['Normal','Osteoporosis']
print(classification_report(test_generator.classes, y_pred_vgg19, target_names=target_names))

#Evaluating model:
x, y = zip(*(test_generator[i] for i in range(len(test_generator))))
x_test, y_test = np.vstack(x), np.vstack(y)
loss_vgg19, acc_vgg19 = model_vgg19.evaluate(x_test, y_test, batch_size=32)

print("Accuracy: " ,acc_vgg19)
print("Loss: ", loss_vgg19)

In [ ]:
import tensorflow as tf
from tensorflow import lite


vgg19 = tf.keras.models.load_model('/content/drive/MyDrive/Hasil Training/Adamax200.hdf5')
converter = tf.lite.TFLiteConverter.from_keras_model(vgg19)
converter.experimental_new_converter = True
vgg19 = converter.convert()
file = open( '/content/drive/MyDrive/Hasil Training/Adamax200.tflite' , 'wb' )
file.write( vgg19 )
print(" vgg19 berhasil di convert")